In [ ]:
# 📘 FX Reinforcement Learning System（簡易版）
"""
- 作成者：fxinsightslab
- バージョン：v1.0
- 公開日：2025年4月xx日
- ライセンス：個人利用限定（再配布・商用利用・改変配布禁止）

---

## 📄 利用に関する注意事項

- このノートブックは学習・研究・検証目的で提供されています。
- 実際の売買判断やトレード運用は自己責任で行ってください。
- 本テンプレートを改変しての再配布、または第三者への提供・販売は禁止です。

> 💬 ご質問や不具合の報告は、X（旧Twitter）[@fxinsightslab](https://x.com/fxinsightslab) までお気軽にどうぞ！

---

✅ このノートはデータ準備用です。　追加したい特徴量があればこのノートに追加してください。
"""

In [1]:
import pandas as pd
import numpy as np
import talib
from sklearn.preprocessing import StandardScaler

# --- シンプルなジグザグ計算（方向のみ：上昇→1, 下降→-1, 変化なし→0）---
def calculate_zigzag(df, deviation=5):
    zz = [0] * len(df)  # 初期化：すべて0
    last_pivot_idx = 0
    last_pivot_price = df["close"].iloc[0]
    direction = 0  # 0=未確定, 1=上昇, -1=下降

    for i in range(1, len(df)):
        price = df["close"].iloc[i]
        change_pct = (price - last_pivot_price) / last_pivot_price * 100

        if direction == 0:
            if abs(change_pct) > deviation:
                direction = 1 if change_pct > 0 else -1
                zz[i] = direction
                last_pivot_idx = i
                last_pivot_price = price
        elif direction == 1:
            if price > last_pivot_price:
                last_pivot_idx = i
                last_pivot_price = price
            elif (last_pivot_price - price) / last_pivot_price * 100 > deviation:
                direction = -1
                zz[i] = direction
                last_pivot_idx = i
                last_pivot_price = price
        elif direction == -1:
            if price < last_pivot_price:
                last_pivot_idx = i
                last_pivot_price = price
            elif (price - last_pivot_price) / last_pivot_price * 100 > deviation:
                direction = 1
                zz[i] = direction
                last_pivot_idx = i
                last_pivot_price = price

    return zz

# --- 拡張ジグザグ：1〜4のシグナルを返す（上昇中/下降中 + 高値/安値ブレイク）---
def calculate_zigzag_trend_signal_with_breaks(df, depth=12, deviation=0.3):
    prices = df["close"].values
    signal = [0] * len(prices)

    last_pivot_price = prices[0]
    last_pivot_index = 0
    direction = 0
    waiting = True

    for i in range(depth, len(prices)):
        price = prices[i]
        price_change_pct = (price - last_pivot_price) / last_pivot_price * 100

        if waiting:
            if abs(price_change_pct) > deviation:
                direction = 1 if price_change_pct > 0 else -1
                last_pivot_price = price
                last_pivot_index = i
                waiting = False
            continue

        if direction == 1:
            if price > last_pivot_price:
                signal[i] = 3  # 高値更新
                last_pivot_price = price
                last_pivot_index = i
            elif (last_pivot_price - price) / last_pivot_price * 100 > deviation:
                direction = -1
                signal[i] = 2  # トレンド転換 → 下降
                last_pivot_price = price
                last_pivot_index = i
            else:
                signal[i] = 1  # 上昇中（ブレイクなし）
        elif direction == -1:
            if price < last_pivot_price:
                signal[i] = 4  # 安値更新
                last_pivot_price = price
                last_pivot_index = i
            elif (price - last_pivot_price) / last_pivot_price * 100 > deviation:
                direction = 1
                signal[i] = 1  # トレンド転換 → 上昇
                last_pivot_price = price
                last_pivot_index = i
            else:
                signal[i] = 2  # 下降中（ブレイクなし）

    return signal

# --- データの前処理関数（テクニカル指標＋ジグザグ＋スケーリング）---
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    # --- 出来高処理（tick_volume or volumes列）---
    df["volume"] = df.get("tick_volume", df.get("volumes", 0))
    
    # --- 元の終値を保存（後で報酬などに使用）---
    df["raw_close"] = df["close"]

    # --- 各種テクニカル指標の計算 ---
    df["SMA_20"] = talib.SMA(df["close"], timeperiod=20)
    df["EMA_50"] = talib.EMA(df["close"], timeperiod=50)
    df["RSI_14"] = talib.RSI(df["close"], timeperiod=14)
    macd, macdsignal, _ = talib.MACD(df["close"])
    df["MACD"] = macd
    df["ATR_14"] = talib.ATR(df["high"], df["low"], df["close"], timeperiod=14)
    upper, middle, lower = talib.BBANDS(df["close"])
    df["BB_upper"] = upper
    df["BB_lower"] = lower
    df["OBV"] = talib.OBV(df["close"], df["volume"])

    # --- ジグザグ指標の追加 ---
    df["zigzag_dir"] = calculate_zigzag(df, deviation=5)  # 上昇/下降フラグ（±1）
    df["zigzag_signal"] = calculate_zigzag_trend_signal_with_breaks(df, depth=12, deviation=0.3)  # 1〜4のトレンド信号

    # --- 欠損値を除去し、安全なコピーを作成 ---
    df = df.dropna().copy()

    # --- 特徴量のスケーリング（正規化）---
    features = [
        "close", "SMA_20", "RSI_14", "MACD", "ATR_14",
        "BB_upper", "BB_lower", "volume", "open", "high", "low", "EMA_50", "OBV"
    ]
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features])  # 各特徴量を標準化

    return df


In [2]:
#from src.utils.data_preparation import prepare_data
import MetaTrader5 as mt5
import pandas as pd

# --- MT5からデータ取得 ---
mt5.initialize()
symbol = "GOLD"
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 1000)
mt5.shutdown()

df = pd.DataFrame(rates)
df["time"] = pd.to_datetime(df["time"], unit="s")

# --- 前処理実行 ---
df = prepare_data(df)


In [3]:
df

,time,open,high,low,close,tick_volume,spread,real_volume,volume,raw_close,SMA_20,EMA_50,RSI_14,MACD,ATR_14,BB_upper,BB_lower,OBV,zigzag_dir,zigzag_signal
49,2025-04-21 07:10:00,0.060735,0.017175,0.036981,-0.009488,647,34,0,-0.705498,3373.83,0.117436,-0.013810,-1.001134,0.276339,-1.271535,0.089124,0.061951,-2.417507,0,2
50,2025-04-21 07:15:00,-0.010621,-0.039363,-0.038161,-0.031765,844,33,0,-0.288342,3372.54,0.107682,-0.015431,-1.179007,0.136453,-1.183211,0.056312,0.023265,-2.499409,0,4
51,2025-04-21 07:20:00,-0.035155,-0.033138,0.010095,0.011234,418,34,0,-1.190416,3375.03,0.101037,-0.015228,-0.626425,0.068278,-1.196536,-0.015945,0.044966,-2.458846,0,2
52,2025-04-21 07:25:00,0.010457,-0.007031,0.025262,-0.000854,510,30,0,-0.995602,3374.33,0.093542,-0.015528,-0.734861,0.003139,-1.184292,-0.036427,0.043631,-2.508337,0,2
53,2025-04-21 07:30:00,-0.002847,-0.035732,0.030777,0.003981,232,33,0,-1.584279,3374.61,0.085442,-0.015618,-0.672308,-0.042312,-1.250288,-0.080841,0.064720,-2.485823,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2025-04-24 17:00:00,-1.021875,-1.016578,-1.044309,-1.077394,1576,27,0,1.261700,3311.99,-0.804495,-0.813859,-1.949650,-1.161366,0.002736,-1.086759,-0.979114,-0.463202,0,4
996,2025-04-24 17:05:00,-1.076990,-1.116340,-1.121347,-1.125229,1776,27,0,1.685208,3309.22,-0.828740,-0.828884,-2.105966,-1.278771,0.021922,-1.079274,-1.035277,-0.635546,0,4
997,2025-04-24 17:10:00,-1.127095,-1.062569,-1.096529,-1.049591,1615,27,0,1.344284,3313.60,-0.852967,-0.840222,-1.412815,-1.281254,0.104989,-1.095561,-1.033066,-0.478825,0,2
998,2025-04-24 17:15:00,-1.047618,-1.082279,-1.086361,-1.118321,1323,27,0,0.725961,3309.62,-0.882352,-0.853930,-1.674350,-1.334993,0.114526,-1.107124,-1.055330,-0.607210,0,2
